 #  A Whale off the Port(folio)
 ---

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P TSX 60 Index.

In [ ]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path

%matplotlib inline

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:

* `whale_returns.csv`: Contains returns of some famous "whale" investors' portfolios.

* `algo_returns.csv`: Contains returns from the in-house trading algorithms from Harold's company.

* `sp_tsx_history.csv`: Contains historical closing prices of the S&P TSX 60 Index.

## Whale Returns

Read the Whale Portfolio daily returns and clean the data.

In [ ]:
csvpath_whale_returns = Path("C:/Users/dude/Downloads/Starter_Code_CH4/Starter_Code/Resources/whale_returns.csv")

In [ ]:
# Reading whale returns
whale_returns_df = pd.read_csv(csvpath_whale_returns)
whale_returns_df.head()

In [ ]:
# Count nulls
whale_returns_df.isnull().mean()*100

In [ ]:
# Drop nulls
whale_returns_df = whale_returns_df.dropna().copy() 

## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data.

In [ ]:
# Reading algorithmic returns
algo_df = pd.read_csv(csvpath_algo)

In [ ]:
# Count nulls
algo_df.isnull().mean()*100

In [ ]:
# Drop nulls
algo_df= algo_df.dropna().copy()

## S&P TSX 60 Returns

Read the S&P TSX 60 historic closing prices and create a new daily returns DataFrame from the data. 

In [ ]:
# Reading S&P TSX 60 Closing Prices
csvpath_sp_df= pd.read_csv (csvpath_sp)

In [ ]:
# Check Data Types
csvpath_sp_df.dtypes

In [ ]:
# Fix Data Types
csvpath_sp_df['Close']= csvpath_sp_df['Close'].str.replace(r'[^A-Za-z0-9]+', '')
csvpath_sp_df["Date"] = csvpath_sp_df["Date"].astype('object')
csvpath_sp_df["Close"] = csvpath_sp_df["Close"].astype('int64')
csvpath_sp_df.dtypes

In [ ]:
# Calculate Daily Returns
csvpath_sp_df = csvpath_sp_df.set_index(pd.to_datetime(csvpath_sp_df["Date"], infer_datetime_format=True))
csvpath_sp_df = csvpath_sp_df.drop(columns=[" Date "])
csvpath_sp_df.head()
daily_return = csvpath_sp_df.pct_change()
daily_return.head()

In [ ]:
# Drop nulls
daily_return_df= daily_return.dropna().copy()
daily_return_df.head()

In [ ]:
# Rename `Close` Column to be specific to this portfolio.
columns = ["Daily_return"]
daily_return_df.columns = columns
daily_return_df.head()

## Combine Whale, Algorithmic, and S&P TSX 60 Returns

In [ ]:
# Join Whale Returns, Algorithmic Returns, and the S&P TSX 60 Returns into a single DataFrame with columns for each portfolio's returns.
combined_data = pd.concat([whale_returns_df, algo_df, daily_return_df], axis="columns", join="inner")
combined_data.head()

---

# Conduct Quantitative Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance Anlysis

#### Calculate and Plot the daily returns.

In [ ]:
# Plot daily returns of all portfolios
whale_returns_df.plot(title="Daily return whale")
algo_df.plot(title="Daily return algo")
daily_return_df.plot(title="Daily return S&P")


#### Calculate and Plot cumulative returns.

In [ ]:
# Calculate cumulative returns of all portfolios
whale_returns_df.set_index(pd.to_datetime(whale_returns_df['date'], infer_datetime_format=True), inplace=True)
whale_returns_df.head()
whale_returns_df.drop(columns=['Date'], inplace=True)
whale_returns_df.head()
algo_df.set_index(pd.to_datetime(algo_df['Date'], infer_datetime_format=True), inplace=True)
algo_df.head()
algo_df.drop(columns=['Date'], inplace=True)
algo_df.head()
daily_return_df.set_index(pd.to_datetime(daily_return_df['Date'], infer_datetime_format=True), inplace=True)
daily_return_df.head()
daily_return_df.drop(columns=['Date'], inplace=True)
daily_return_df.head()

cumulative_returns_whale = (1 + whale_returns_df).cumprod() - 1
cumulative_returns_whale
cumulative_algo_df = (1 + algo_df).cumprod() - 1
cumulative_algo_df
cumulative_daily_return_df = (1 + daily_return_df).cumprod() - 1
cumulative_daily_return_df

# Plot cumulative returns
cumulative_returns_whale.plot(figsize=(10,5), title="Cumulative Returns of Whale over the Last 10 Years")
cumulative_algo_df.plot(figsize=(10,5), title="Cumulative Returns of algo over the Last 10 Years")
cumulative_daily_return_df.plot(figsize=(10,5), title="Cumulative Returns of S&P over the Last 10 Years")


---

## Risk Analysis

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios.
4. Determine which portfolios are riskier than the S&P TSX 60.
5. Calculate the Annualized Standard Deviation.

### Create a box plot for each portfolio


In [ ]:
# Box plot to visually show risk
algo_df.plot.box(figsize=(10,5), title="algo over the Last 10 Years")
whale_df.plot.box(figsize=(10,5), title="whale over the Last 10 Years")
daily_return_df.plot.box(figsize=(10,5), title="S&P over the Last 10 Years")

### Calculate Standard Deviations

In [ ]:
# Calculate the daily standard deviations of all portfolios
# Calculate std dev
algo_df_std = algo_df.std()
algo_df_std.head()
whale_returns_df_std = whale_returns_df.std()
whale_returns_df_std.head()


### Determine which portfolios are riskier than the S&P TSX 60

In [ ]:
# Calculate  the daily standard deviation of S&P TSX 60
daily_return_df_std = daily_return_df.std()
daily_return_df_std.head()
# Determine which portfolios are riskier than the S&P TSX 60
#answere: whale portfolios Tiger and Berkshier are reskier than the S&P TXS 60 

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized standard deviation (252 trading days)
daily_return_df_std_annual = daily_return_df.std() * np.sqrt(252)
whale_returns_df_std_annual = whale_returns_df.std() * np.sqrt(252)
algo_df_std_annual = algo_df.std() * np.sqrt(252)
algo_df_std_annual
whale_returns_df_std_annual
daily_return_df_std_annual.head()

---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Calculate and plot the rolling standard deviation for all portfolios using a 21-day window.
2. Calculate the correlation between each stock to determine which portfolios may mimick the S&P TSX 60.
3. Choose one portfolio, then calculate and plot the 60-day rolling beta for it and the S&P TSX 60.

### Calculate and plot rolling `std` for all portfolios with 21-day window

In [ ]:
# Calculate the rolling standard deviation for all portfolios using a 21-day window
algo_ma_21 = algo_df.rolling(window=21).mean()
algo_ma_21
whale_returns_ma_21 = whale_returns_df.rolling(window=21).mean()
whale_returns_ma_21
# Plot the rolling standard deviation
algo = algo_df.plot(figsize=(25,10))
whale = whale_returns_df.plot(figsize=(25,10))

# Overlay STD20, STD50, and STD100 on the same figure
algo_df.rolling(window=21).std().plot(ax=algo)
whale_returns_df.rolling(window=51).std().plot(ax=whale)
# Set the legend of the figure
algo.legend([ "AlgoMA21", "whaleMA21"]);
whale.legend([ "AlgoMA21", "whaleMA21"]);

### Calculate and plot the correlation

In [ ]:
# Calculate the correlation
correlation_algo = algo_df.corr()
correlation_algo
correlation_whales = whale_returns_df.corr()
correlation_whales
# Display de correlation matrix
import seaborn as sns
sns.heatmap(correlation_whales)
sns.heatmap(correlation_algo)

### Calculate and Plot Beta for a chosen portfolio and the S&P 60 TSX

In [ ]:
# Calculate covariance of a single portfolio
SOROS_covariance = whale_returns_df["SOROS FUND MANAGEMENT LLC"].cov(daily_return_df["Daily_return"])
print(f"SOROS Covariance: {SOROS_covariance}")

# Calculate variance of S&P TSX
# Calculate variance of all daily returns of . S&P 500
variance = daily_return_df["Daily_return"].var()

# Diplay S&P 500 variance
print(f"S&P 500 variance: {variance}")
# Computing beta
# Calculate beta of all daily returns stocks
SOROS_beta = SOROS_covariance / variance


# Display the beta value of each social media stock
print(f"SOROS: {SOROS_beta}")
# Plot beta trend
ax = SOROS_beta.plot(figsize=(20, 10), title="SOROS_beta Beta ")
SOROS_beta.plot(ax=ax)

## Rolling Statistics Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the [`ewm`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) with a 21-day half life for each portfolio, using standard deviation (`std`) as the metric of interest.

In [ ]:
# Use `ewm` to calculate the rolling window
algo_ema_21 = algo_df.ewm(span=127, adjust=False).mean()
algo_ema = algo_df.plot(figsize=(25,10))

algo_ema_21.rolling(window=21).plot(ax=algo_ema)

---

# Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. After all, if you could invest in one of two portfolios, and each offered the same 10% return, yet one offered lower risk, you'd take that one, right?

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Annualized Sharpe Ratios
# Calculate sharpe ratios
whale_sharpe_ratios = (whale_returns_df.mean() * 252) / (whale_returns_df_std_annual)
whale_sharpe_ratios
algo_sharpe_ratios = (algo_df.mean() * 252) / (algo_df_std_annual)
algo_sharpe_ratios
daily_return_sharpe_ratios = (daily_return_df.mean() * 252) / (daily_return_df_std_annual)
daily_return_sharpe_ratios

In [ ]:
# Visualize the sharpe ratios as a bar plot
algo_sharpe_ratios.plot.bar(title="Sharpe Ratios") 
whale_sharpe_ratios.plot.bar(title="Sharpe Ratios") 
daily_return_sharpe_ratios.plot.bar(title="Sharpe Ratios") 

### Determine whether the algorithmic strategies outperform both the market (S&P TSX 60) and the whales portfolios.

Write your answer here!

yes the algorethmic stratgies outperform both the S&P and the whales portfolios and it is clear from the sharp ratios

---

# Create Custom Portfolio

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P TSX 60. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock.
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns.
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others.
5. Include correlation analysis to determine which stocks (if any) are correlated.

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

In [ ]:
# Reading data from 1st stock
google_df = pd.read_csv(google_Path)
google_df.head()

In [ ]:
# Reading data from 2nd stock
amazon_df = pd.read_csv(Amazon_path)
amazon_df.head()

In [ ]:
# Reading data from 3rd stock
TSLA_df = pd.read_csv(TSLA_path)
TSLA_df.head()

In [ ]:
# Combine all stocks in a single DataFrame
My_data = pd.concat([TSLA_df, amazon_df, google_df], axis="columns", join="inner")
My_data.head()
combined_data = pd.concat([whale_returns_df, algo_df, daily_return_df,TSLA_df, amazon_df, google_df], axis="columns", join="inner")
combined_data.head()

In [ ]:
# Reset Date index
TSLA_df = TSLA_df.set_index(pd.to_datetime(TSLA_df["Date"], infer_datetime_format=True))
amazon_df = amazon_df.set_index(pd.to_datetime(amazon_df["Date"], infer_datetime_format=True))
google_df = google_df.set_index(pd.to_datetime(google_df["Date"], infer_datetime_format=True))
amazon_df = amazon_df.drop(columns=["Date"])
google_df = google_df.drop(columns=["Date"])
TSLA_df = TSLA_df.drop(columns=["Date"])


In [ ]:
# Reorganize portfolio data by having a column per symbol


In [ ]:
# Calculate daily returns

# Drop NAs

# Display sample data


## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Set weights
weights = [1/3, 1/3, 1/3]

# Calculate portfolio return

# Display sample data


## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [ ]:
# Join your returns DataFrame to the original returns DataFrame


In [ ]:
# Only compare dates where return data exists for all the stocks (drop NaNs)


## Re-run the risk analysis with your portfolio to see how it compares to the others

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized `std`


### Calculate and plot rolling `std` with 21-day window

In [ ]:
# Calculate rolling standard deviation

# Plot rolling standard deviation


### Calculate and plot the correlation

In [ ]:
# Calculate and plot the correlation


### Calculate and Plot the 60-day Rolling Beta for Your Portfolio compared to the S&P 60 TSX

In [ ]:
# Calculate and plot Beta


### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Calculate Annualized Sharpe Ratios


In [ ]:
# Visualize the sharpe ratios as a bar plot


### How does your portfolio do?

Write your answer here!